In [9]:
%matplotlib notebook

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

from scipy.stats import linregress

# Import API key

# File to Load
profiles_path = "Resources/okcupid_profiles_cut.csv"

# Read the modified Comic Books csv and store into Pandas DataFrame
profiles_df = pd.read_csv(profiles_path, encoding="UTF-8")
profiles_df.head()




C:\Users\jlper\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (0,6,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,age,sex,orientation,drinks,drugs,education,income,job,location,sign,...,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142
0,22,m,straight,socially,never,working on college/university,-1,transportation,"south san francisco, california",gemini,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,35,m,straight,often,sometimes,working on space camp,80000,hospitality / travel,"oakland, california",cancer,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,38,m,straight,socially,NaN,graduated from masters program,-1,NaN,"san francisco, california",pisces but it doesn&rsquo;t matter,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23,m,straight,socially,NaN,working on college/university,20000,student,"berkeley, california",pisces,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29,m,straight,socially,never,graduated from college/university,-1,artistic / musical / writer,"san francisco, california",aquarius,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Extract only columns that will be needed

reduced_profiles_df = profiles_df.loc[:, ["age", "sex", "orientation",
                                    "drinks", "drugs", "education", "income", "job", "location", "sign"]]
reduced_profiles_df.head()


,age,sex,orientation,drinks,drugs,education,income,job,location,sign
0,22,m,straight,socially,never,working on college/university,-1,transportation,"south san francisco, california",gemini
1,35,m,straight,often,sometimes,working on space camp,80000,hospitality / travel,"oakland, california",cancer
2,38,m,straight,socially,NaN,graduated from masters program,-1,NaN,"san francisco, california",pisces but it doesn&rsquo;t matter
3,23,m,straight,socially,NaN,working on college/university,20000,student,"berkeley, california",pisces
4,29,m,straight,socially,never,graduated from college/university,-1,artistic / musical / writer,"san francisco, california",aquarius


In [12]:
# Split 'location' into City and State into diffent columns

reduced_profiles_df[['City', 'State']]= reduced_profiles_df['location'].str.split(',', n=1, expand=True)
reduced_profiles_df.head()


,age,sex,orientation,drinks,drugs,education,income,job,location,sign,City,State
0,22,m,straight,socially,never,working on college/university,-1,transportation,"south san francisco, california",gemini,south san francisco,california
1,35,m,straight,often,sometimes,working on space camp,80000,hospitality / travel,"oakland, california",cancer,oakland,california
2,38,m,straight,socially,NaN,graduated from masters program,-1,NaN,"san francisco, california",pisces but it doesn&rsquo;t matter,san francisco,california
3,23,m,straight,socially,NaN,working on college/university,20000,student,"berkeley, california",pisces,berkeley,california
4,29,m,straight,socially,never,graduated from college/university,-1,artistic / musical / writer,"san francisco, california",aquarius,san francisco,california


In [13]:
# Combining similar values under 'sign' column

reduced_profiles_df = reduced_profiles_df.replace({
    "aquarius but it doesn&rsquo;t matter": "aquarius", "aquarius and it&rsquo;s fun to think about": "aquarius", "aquarius and it matters a lot": "aquarius",
    "aries and it matters a lot": "aries", "aries and it&rsquo;s fun to think about": "aries", "aries but it doesn&rsquo;t matter": "aries",
    "cancer but it doesn&rsquo;t matter": "cancer", "cancer and it matters a lot": "cancer", "cancer and it&rsquo;s fun to think about": "cancer",
    "capricorn and it matters a lot": "capricorn", "capricorn and it&rsquo;s fun to think about": "capricorn", "capricorn but it doesn&rsquo;t matter": "capricorn",
    "gemini and it matters a lot": "gemini", "gemini and it&rsquo;s fun to think about": 'gemini', "gemini but it doesn&rsquo;t matter": 'gemini',
    "leo and it matters a lot": "leo", "leo and it&rsquo;s fun to think about": "leo", "leo but it doesn&rsquo;t matter": "leo",
    "libra and it matters a lot": "libra", "libra and it&rsquo;s fun to think about": "libra", "libra but it doesn&rsquo;t matter": "libra",
    "pisces and it matters a lot": "pisces", "pisces and it&rsquo;s fun to think about": "pisces", "pisces but it doesn&rsquo;t matter": "pisces",
    "sagittarius and it&rsquo;s fun to think about": "sagittarius", "sagittarius and it matters a lot": "sagittarius", "sagittarius but it doesn&rsquo;t matter": "sagittarius",
    "scorpio and it matters a lot": "scorpio", "scorpio and it&rsquo;s fun to think about": "scorpio", "scorpio but it doesn&rsquo;t matter": "scorpio",
    "taurus but it doesn&rsquo;t matter": "taurus", "taurus and it&rsquo;s fun to think about": "taurus", "taurus and it matters a lot": "taurus",
    "virgo but it doesn&rsquo;t matter": "virgo", "virgo and it&rsquo;s fun to think about": "virgo", "virgo and it matters a lot": "virgo"

})
reduced_profiles_df.head()

,age,sex,orientation,drinks,drugs,education,income,job,location,sign,City,State
0,22,m,straight,socially,never,working on college/university,-1,transportation,"south san francisco, california",gemini,south san francisco,california
1,35,m,straight,often,sometimes,working on space camp,80000,hospitality / travel,"oakland, california",cancer,oakland,california
2,38,m,straight,socially,NaN,graduated from masters program,-1,NaN,"san francisco, california",pisces,san francisco,california
3,23,m,straight,socially,NaN,working on college/university,20000,student,"berkeley, california",pisces,berkeley,california
4,29,m,straight,socially,never,graduated from college/university,-1,artistic / musical / writer,"san francisco, california",aquarius,san francisco,california


In [14]:
reduced_profiles_df["sign"].value_counts()

leo                                                                                                             4374
gemini                                                                                                          4310
libra                                                                                                           4207
cancer                                                                                                          4206
virgo                                                                                                           4141
taurus                                                                                                          4140
scorpio                                                                                                         4134
aries                                                                                                           3989
pisces                                                          

In [15]:
# Create the bins in which Data will be held for age groups

bins = [0, 19.9, 29.9, 39.9, 49.9, 59.9, 69.9, 79.9, 89.9, 99.9, 110]

# Create the years for the ten bins

group_years = ["Teens", "In their 20's", "In their 30's", "In their 40's", "In their 50's", 
                "In their 60's", "In their 70's", "In their 80's", "In their 90's", "Above 100yrs"]

In [16]:
#Add new column for 'Age Group Summary'

reduced_profiles_df["Age Group Summary"] = pd.cut(reduced_profiles_df["age"], bins, labels=group_years, include_lowest=True)
reduced_profiles_df

TypeError: '<' not supported between instances of 'int' and 'str'

In [ ]:
#grouped_by_sex_df = reduced_profiles_df.groupby(['sex'])
#grouped_by_sex_df.count()

In [ ]:
reduced_profiles_df.count()

In [ ]:
#Separate all information for only 'm' (male) under "sex" column

males_df = reduced_profiles_df.loc[(reduced_profiles_df["sex"]== "m")]
males_df.head()

In [ ]:
males = len(males_df)
males

In [ ]:
#Separate all information for only 'f' (female) under "sex" column

females_df = reduced_profiles_df.loc[(reduced_profiles_df["sex"]== "f")]
females_df.head()

In [ ]:
females = len(females_df)
females

In [ ]:
gender = ["Male", "Female"]
gender_counts = [35829, 24117]
x_axis = np.arange(len(gender_counts))

In [ ]:
# Create a bar chart based upon the above data
plt.bar(x_axis, gender_counts, color='b', alpha=0.5, align="center")

In [ ]:
# Create the ticks for our bar chart's x axis
tick_locations = [value for value in x_axis]
plt.xticks(tick_locations, gender)

In [ ]:
# Set the limits of the x axis
plt.xlim(-0.75, len(x_axis)-0.25)

In [ ]:
# Set the limits of the y axis
plt.ylim(0, max(gender_counts)+2000)


In [ ]:
# Give the chart a title, x label, and y label
plt.title("OkCupid Male vs. Female User")
plt.xlabel("Gender")
plt.ylabel("Number of Users")


In [ ]:
# Save an image of the chart and print it to the screen
plt.savefig("images_estela/gender_distribution.png")
plt.show(block=False)

reduced_profiles_df = reduced_profiles_df.dropna(how='any')
len(reduced_profiles_df)

for i in range(len(reduced_profiles_df)):
    print (i,reduced_profiles_df.iloc[i,:])
    num=pd.to_numeric(reduced_profiles_df.iloc[i,6])

# Use pd.to_numeric() method to convert the datatype of the Amount column
#reduced_profiles_df['income'] = pd.to_numeric(reduced_profiles_df['income'])